In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.13.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl tensorboard-plugin-profile

In [ ]:
!pip install google-api-python-client==1.12.1

In [ ]:
!pip install pytorch-lightning bert_score datasets rouge_score evaluate transformers wandb torchmetrics deepspeed

In [ ]:
!git clone https://github.com/Myashka/CQA_RLHF.git cqa

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
import yaml

In [ ]:
trainer_config = dict(
    seed=42,
    model_name="EleutherAI/gpt-neo-125M",
    wandb=dict(
        api="",
        project_name="CQA_RLHF",
        args=dict(job_type="train", group="sft", name="125M-freezed-cuda"),
    ),
    data=dict(
        data_dir=r"/content/drive/MyDrive/Diploma/data/1.0-data-div-ans-sep.json",
        max_length=512,
        batch_size=16,
        pad_for_tpu = False
    ),
    model_params=dict(
        learning_rate=2e-5,
        do_freeze=True,
        use_cache=False,
        warmup_steps=100,
        adam_betas=[0.9, 0.95],
        weight_decay=0.001,
        do_compute_metrics = False,
        do_compute_bertscore = False
    ),
    trainer=dict(
        checkpoint=dict(
            every_n_train_steps=100,
            dirpath=r"/content/drive/MyDrive/Diploma/Checkpoints",
        ),
        ckpt_path=None,
        params=dict(
            accelerator="auto",
            max_epochs=3,
            auto_scale_batch_size=True,
            accumulate_grad_batches=1,
            gradient_clip_val=1,
            precision="16",
            val_check_interval=50,
            overfit_batches=0,  # 0 for train
            num_sanity_val_steps=2,
            log_every_n_steps = 1,
            limit_train_batches=100,
            strategy="deepspeed_stage_2_offload"
        ),
    ),
)

with open("trainer_config.yaml", "w") as outfile:
    yaml.dump(trainer_config, outfile, default_flow_style=False)

In [ ]:
!python /content/cqa/sft/lightning/pl_train.py --config_file /content/trainer_config.yaml